# Lab assignment №1, part 2

This lab assignment consists of several parts. You are supposed to make some transformations, train some models, estimate the quality of the models and explain your results.

Several comments:
* Don't hesitate to ask questions, it's a good practice.
* No private/public sharing, please. The copied assignments will be graded with 0 points.
* Blocks of this lab will be graded separately.

__*This is the second part of the assignment. First and third parts are waiting for you in the same directory.*__

## Part 2. Data preprocessing, model training and evaluation.

In [1]:
import warnings
warnings.filterwarnings('ignore')

### 1. Reading the data
Today we work with the [dataset](https://archive.ics.uci.edu/ml/datasets/Statlog+%28Vehicle+Silhouettes%29), describing different cars for multiclass ($k=4$) classification problem. The data is available below.

In [2]:
# If on colab, uncomment the following lines

! wget https://raw.githubusercontent.com/girafe-ai/ml-course/22f_basic/homeworks/lab01_ml_pipeline/car_data.csv

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


dataset = pd.read_csv('car_data.csv', delimiter=',', header=None).values
data = dataset[:, :-1].astype(int)
target = dataset[:, -1]

print(data.shape, target.shape)

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.35)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'car_data.csv'

To get some insights about the dataset, `pandas` might be used. The `train` part is transformed to `pd.DataFrame` below.

In [ ]:
X_train_pd = pd.DataFrame(X_train)

# First 15 rows of our dataset.
X_train_pd.head(15)

Methods `describe` and `info` deliver some useful information.

In [ ]:
X_train_pd.describe()

In [ ]:
X_train_pd.info()

### 2. Machine Learning pipeline
Here you are supposed to perform the desired transformations. Please, explain your results briefly after each task.

#### 2.0. Data preprocessing
* Make some transformations of the dataset (if necessary). Briefly explain the transformations

Firstly, the first feature is index. So let's delete it.

In [ ]:
### YOUR CODE HERE
X_train = np.delete(X_train, 0, 1)
X_test = np.delete(X_test, 0, 1)

It's worth trying to normalize the data, even though the variation of scales is not so large between the features.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(16, 16))
sns.heatmap(pd.DataFrame(X_train).corr().abs(), annot=True)

Okay, 7-12 are correlated (look at the heatmap). I've also tried to get rid of some strongly correlated features (I made made some plots, etc.), but this did not improve much.



In [ ]:
X_train = np.delete(X_train, 11, 1)
X_test = np.delete(X_test, 11, 1)

In [ ]:
print(X_train.shape, X_test.shape)

#### 2.1. Basic logistic regression
* Find optimal hyperparameters for logistic regression with cross-validation on the `train` data (small grid/random search is enough, no need to find the *best* parameters).

* Estimate the model quality with `f1` and `accuracy` scores.
* Plot a ROC-curve for the trained model. For the multiclass case you might use `scikitplot` library (e.g. `scikitplot.metrics.plot_roc(test_labels, predicted_proba)`).

*Note: please, use the following hyperparameters for logistic regression: `multi_class='multinomial'`, `solver='saga'` `tol=1e-3` and ` max_iter=500`.*

In [ ]:
### YOUR CODE HERE
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV


# Define model
model = LogisticRegression(tol=1e-3, solver='saga', max_iter=500, multi_class='multinomial')

# Define evaluation
cv = RepeatedStratifiedKFold(random_state=1)

# Define search space; C: inverse of reg strength
space = dict()
space['penalty'] = ['l1', 'l2', 'elasticnet']
space['C'] = list(np.logspace(-5, 5, 10))

# Define search
search = GridSearchCV(estimator=model, param_grid=space, scoring='accuracy', n_jobs=-1, cv=cv)

# Execute search
logCV = search.fit(X_train, y_train)

# Results of search
print(f"Best Accuracy: {logCV.best_score_}")
print(f"Best Params: {logCV.best_params_}")

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

# Train our model and predict labels
model = LogisticRegression(tol=1e-3, solver='saga', max_iter=500, multi_class='multinomial', C=logCV.best_params_['C'], penalty=logCV.best_params_['penalty'])
clf = model.fit(X_train, y_train)
y_pred = clf.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, None]
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f"F1-macro: {f1_score(y_test, y_pred, average='macro')}")
print(f'F1: {f1_score(y_test, y_pred, average=None)}')
sns.heatmap(cm, annot=True)
plt.show()

Classes 1 and 2 were poorly classified (among themselves in particular). I think I know the reason: we classify a bus, a van and two "almost the same" passenger cars.

In [ ]:
!pip install scikit-plot

In [ ]:
from scikitplot.metrics import plot_roc

probas = clf.predict_proba(X_test)
plot_roc(y_test, probas)
plt.show()

#### 2.2. PCA: explained variance plot
* Apply the PCA to the train part of the data. Build the explaided variance plot. 

In [ ]:
### YOUR CODE HERE
from sklearn.decomposition import PCA

# Data is already normalized
pca = PCA()
pca.fit_transform(X_train)

expl_var = pca.explained_variance_
fig, ax = plt.subplots(figsize=(12, 8))
ax.plot(range(1, len(expl_var) + 1), expl_var)
ax.bar(range(1, len(expl_var) + 1), expl_var, alpha=0.5, color='grey')
ax.set_xticks(range(1, 19))
plt.show()

#### 2.3. PCA trasformation
* Select the appropriate number of components. Briefly explain your choice. Should you normalize the data?

*Use `fit` and `transform` methods to transform the `train` and `test` parts.*

The features after 10 give a small contribution in total. 

In [ ]:
### YOUR CODE HERE
# Data is (are) already normalized
pca = PCA(n_components=10)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

**Note: From this point `sklearn` [Pipeline](https://scikit-learn.org/stable/modules/compose.html) might be useful to perform transformations on the data. Refer to the [docs](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) for more information.**

#### 2.4. Logistic regression on PCA-preprocessed data.
* Find optimal hyperparameters for logistic regression with cross-validation on the transformed by PCA `train` data.

* Estimate the model quality with `f1` and `accuracy` scores.
* Plot a ROC-curve for the trained model. For the multiclass case you might use `scikitplot` library (e.g. `scikitplot.metrics.plot_roc(test_labels, predicted_proba)`).

*Note: please, use the following hyperparameters for logistic regression: `multi_class='multinomial'`, `solver='saga'` and `tol=1e-3`*

In [ ]:
### YOUR CODE HERE

In [ ]:
# Find optimal hyperparams
model = LogisticRegression(tol=1e-3, solver='saga', max_iter=500, multi_class='multinomial')

cv = RepeatedStratifiedKFold(random_state=1)

space = dict()
space['penalty'] = ['l1', 'l2', 'elasticnet']
space['C'] = list(np.logspace(-5, 5, 10))

search = GridSearchCV(estimator=model, param_grid=space, scoring='accuracy', n_jobs=-1, cv=cv)
logCV_pca = search.fit(X_train_pca, y_train)

In [ ]:
# Train model
model = LogisticRegression(tol=1e-3, solver='saga', max_iter=500, multi_class='multinomial', C=logCV_pca.best_params_['C'], penalty=logCV_pca.best_params_['penalty'])
clf = model.fit(X_train_pca, y_train)
y_pred = clf.predict(X_test_pca)

cm = confusion_matrix(y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, None]
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f"F1-macro: {f1_score(y_test, y_pred, average='macro')}")
print(f'F1: {f1_score(y_test, y_pred, average=None)}')
sns.heatmap(cm, annot=True)

In [ ]:
# Plot ROC-Curve
probas = clf.predict_proba(X_test_pca)
plot_roc(y_test, probas)

#### 2.5. Decision tree
* Now train a desicion tree on the same data. Find optimal tree depth (`max_depth`) using cross-validation.

* Measure the model quality using the same metrics you used above.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# YOUR CODE HERE

# Find optimal depth
model = DecisionTreeClassifier()
cv = RepeatedStratifiedKFold(random_state=1)
space = {'max_depth': range(3, 15)}
search = GridSearchCV(estimator=model, param_grid=space, n_jobs=-1, cv=cv)
treeCV = search.fit(X_train, y_train)

print(treeCV.best_params_)

In [ ]:
model = DecisionTreeClassifier(max_depth=treeCV.best_params_['max_depth'])
clf = model.fit(X_train, y_train)
y_pred = clf.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, None]
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f"F1-macro: {f1_score(y_test, y_pred, average='macro')}")
print(f'F1: {f1_score(y_test, y_pred, average=None)}')
sns.heatmap(cm, annot=True)

In [ ]:
probas = clf.predict_proba(X_test)
plot_roc(y_test, probas)

#### 2.6. Bagging.
Here starts the ensembling part.

First we will use the __Bagging__ approach. Build an ensemble of $N$ algorithms varying N from $N_{min}=2$ to $N_{max}=100$ (with step 5).

We will build two ensembles: of logistic regressions and of decision trees.

*Comment: each ensemble should be constructed from models of the same family, so logistic regressions should not be mixed up with decision trees.*


*Hint 1: To build a __Bagging__ ensebmle varying the ensemble size efficiently you might generate $N_{max}$ subsets of `train` data (of the same size as the original dataset) using bootstrap procedure once. Then you train a new instance of logistic regression/decision tree with optimal hyperparameters you estimated before on each subset (so you train it from scratch). Finally, to get an ensemble of $N$ models you average the $N$ out of $N_{max}$ models predictions.*

*Hint 2: sklearn might help you with this taks. Some appropriate function/class might be out there.*

* Plot `f1` and `accuracy` scores plots w.r.t. the size of the ensemble.

* Briefly analyse the plot. What is the optimal number of algorithms? Explain your answer.

* How do you think, are the hyperparameters for the decision trees you found in 2.5 optimal for trees used in ensemble? 

In [ ]:
# For convinience
!pip install tqdm
from tqdm import tqdm

In [ ]:
# YOUR CODE HERE
from sklearn.ensemble import BaggingClassifier

**Decision Trees Ensemble:**

In [ ]:
# Trying to find the best hyperparameters over the subsets
tree_bag = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100)
cv = RepeatedStratifiedKFold(random_state=1)
space = {'base_estimator__max_depth': range(3, 15)}
search = GridSearchCV(estimator=tree_bag, param_grid=space, n_jobs=-1, cv=cv)
tree_bag_CV = search.fit(X_train, y_train) 
# Check the params
print(tree_bag_CV.best_params_)
# Making optimal base estimator using founded params
opt_base=DecisionTreeClassifier(max_depth=tree_bag_CV.best_params_['base_estimator__max_depth']) 

# Training the model and testing it for size of emsembles 
f1_macro, f1, accuracy = np.zeros(20), np.zeros((20, 4)), np.zeros(20) 
N = np.arange(2, 100, 5)
for _ in tqdm(range(20)):
  for i, n in enumerate(N):  
    tree_bag = BaggingClassifier(base_estimator=opt_base, n_estimators=n)
    tree_bag.fit(X_train, y_train)
    f1_macro[i] = f1_score(y_test, tree_bag.predict(X_test), average='macro')
    f1[i] = f1_score(y_test, tree_bag.predict(X_test), average=None)
    accuracy[i] = accuracy_score(y_test, tree_bag.predict(X_test))

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(28, 8)) 
ax1.plot(N, f1_macro)
ax1.title.set_text('f1_macro')
ax1.set_xticks(N)
ax1.grid()
ax2.plot(N, f1)
ax2.legend(['bus', 'opel', 'saab', 'van'])
ax2.title.set_text('f1_classes')
ax3.plot(N, accuracy)
ax3.title.set_text('accuracy')
plt.show()

Using the bagging we decrease the variance. We need to have a low bias using the ensemble of trees, so the depth of the trees must be enough. But each tree in the ensemble is trained on bootstrapped samples based on the original data. The data is (are) still slightly different. In this case we've got (almost) the same depth.

The optimal number of algorithms is 47.

**Logistic Regression Ensemble**

In [ ]:
log_bag = BaggingClassifier(base_estimator=LogisticRegression(), n_estimators=100)
cv = RepeatedStratifiedKFold(random_state=1)
space = {'base_estimator__C': np.logspace(-5, 5, 10)}
search = GridSearchCV(estimator=log_bag, param_grid=space, n_jobs=-1, cv=cv)
log_bag_CV = search.fit(X_train, y_train) 
# Check the params
print(log_bag_CV.best_params_)
# Making optimal base estimator using founded params
opt_base=LogisticRegression(C=log_bag_CV.best_params_['base_estimator__C'])

f1_macro, f1, accuracy = np.zeros(20), np.zeros((20, 4)), np.zeros(20) 
N = np.arange(2, 100, 5)
for i, n in enumerate(N):
  log_bag = BaggingClassifier(base_estimator=opt_base, n_estimators=n)
  log_bag.fit(X_train, y_train)
  f1_macro[i] = f1_score(y_test, log_bag.predict(X_test), average='macro')
  f1[i] = f1_score(y_test, log_bag.predict(X_test), average=None)
  accuracy[i] = accuracy_score(y_test, log_bag.predict(X_test))

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(28, 8)) 
ax1.plot(N, f1_macro)
ax1.title.set_text('f1_macro')
ax1.set_xticks(N)
ax1.grid()
ax2.plot(N, f1)
ax2.legend(['bus', 'opel', 'saab', 'van'])
ax2.title.set_text('f1_classes')
ax3.plot(N, accuracy)
ax3.title.set_text('accuracy')
plt.show()

The optimal number of algorithms is 52.

#### 2.7. Random Forest
Now we will work with the Random Forest (its `sklearn` implementation).

* * Plot `f1` and `accuracy` scores plots w.r.t. the number of trees in Random Forest.

* What is the optimal number of trees you've got? Is it different from the optimal number of logistic regressions/decision trees in 2.6? Explain the results briefly.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# YOUR CODE HERE
rf = RandomForestClassifier(n_estimators=100, max_features='sqrt')
cv = RepeatedStratifiedKFold(random_state=1)
space = {'max_depth': range(2, 15),
         'criterion': ['gini', 'entropy', 'log_loss']}
search = GridSearchCV(estimator=rf, param_grid=space, n_jobs=-1, cv=cv)
rf_CV = search.fit(X_train, y_train) 

# Training the model and testing it for size of emsembles 
f1_macro, f1, accuracy = np.zeros(20), np.zeros((20, 4)), np.zeros(20) 
N = np.arange(2, 100, 5)
for _ in tqdm(range(20)):
  for i, n in enumerate(N):  
    rf = RandomForestClassifier(n_estimators=n,
                                criterion=rf_CV.best_params_['criterion'],
                                max_depth=rf_CV.best_params_['max_depth'],
                                max_features='sqrt')
    rf.fit(X_train, y_train)
    f1_macro[i] = f1_score(y_test, rf.predict(X_test), average='macro')
    f1[i] = f1_score(y_test, rf.predict(X_test), average=None)
    accuracy[i] = accuracy_score(y_test, rf.predict(X_test))

In [ ]:
print(rf_CV.best_params_)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(28, 8)) 
ax1.plot(N, f1_macro)
ax1.title.set_text('f1_macro')
ax1.set_xticks(N)
ax1.grid()
ax2.plot(N, f1)
ax2.legend(['bus', 'opel', 'saab', 'van'])
ax2.title.set_text('f1_classes')
ax3.plot(N, accuracy)
ax3.title.set_text('accuracy')
plt.show()

The optimal number of algorithms is different from it was on bagging. Random forest uses random features on each node that makes the algorithms less correlated ${\Rightarrow}$ variance decreases. Using random forest we actually obtained the best metrics. 

#### 2.8. Learning curve
Your goal is to estimate, how does the model behaviour change with the increase of the `train` dataset size.

* Split the training data into 10 equal (almost) parts. Then train the models from above (Logistic regression, Desicion Tree, Random Forest) with optimal hyperparameters you have selected on 1 part, 2 parts (combined, so the train size in increased by 2 times), 3 parts and so on.

* Build a plot of `accuracy` and `f1` scores on `test` part, varying the `train` dataset size (so the axes will be score - dataset size.

* Analyse the final plot. Can you make any conlusions using it? 

In [ ]:
# YOUR CODE HERE
f1_macro, f1, accuracy = np.zeros(9), np.zeros((9, 4)), np.zeros(9)
train_size = np.zeros(9)
splits = np.linspace(0, .99, 10)
for i, split in enumerate(splits[1:]):
  # Splitting
  X_train_split, _, y_train_split, _ = train_test_split(X_train, y_train, train_size=split)
  # Best hyperparams:
  estim = LogisticRegression(tol=1e-3, solver='saga', max_iter=500, multi_class='multinomial')
  space = {'penalty': ['l1', 'l2', 'elasticnet'],
           'C': np.logspace(-5, 5, 10)}
  search = GridSearchCV(estimator=estim, param_grid=space, scoring='accuracy', n_jobs=-1, refit=True)
  CV = search.fit(X_train_split, y_train_split) 
  # Fitting the model with the best hyperparams
  model = CV.best_estimator_
  model.fit(X_train_split, y_train_split)

  f1_macro[i] = f1_score(y_test, model.predict(X_test), average='macro')
  f1[i] = f1_score(y_test, model.predict(X_test), average=None)
  accuracy[i] = accuracy_score(y_test, model.predict(X_test))

  train_size[i] = split

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(28, 8)) 
ax1.plot(train_size, f1_macro)
ax1.title.set_text('f1_macro')
ax1.set_xticks(train_size)
ax1.grid()
ax2.plot(train_size, f1)
ax2.legend(['bus', 'opel', 'saab', 'van'])
ax2.title.set_text('f1_classes')
ax3.plot(train_size, accuracy)
ax3.title.set_text('accuracy')
plt.show()

In [ ]:
f1_macro, f1, accuracy = np.zeros(9), np.zeros((9, 4)), np.zeros(9)
train_size = np.zeros(9)
splits = np.linspace(0, .99, 10)
for i, split in enumerate(splits[1:]):
  # Splitting
  X_train_split, _, y_train_split, _ = train_test_split(X_train, y_train, train_size=split)
  # Best hyperparams:
  estim = DecisionTreeClassifier()
  space = {'max_depth': range(3, 15), 'criterion': ['gini', 'entropy']}
  search = GridSearchCV(estimator=estim, param_grid=space, scoring='accuracy', n_jobs=-1, refit=True)
  CV = search.fit(X_train_split, y_train_split) 
  # Fitting the model with the best hyperparams
  model = CV.best_estimator_
  model.fit(X_train_split, y_train_split)

  f1_macro[i] = f1_score(y_test, model.predict(X_test), average='macro')
  f1[i] = f1_score(y_test, model.predict(X_test), average=None)
  accuracy[i] = accuracy_score(y_test, model.predict(X_test))

  train_size[i] = split

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(28, 8)) 
ax1.plot(train_size, f1_macro)
ax1.title.set_text('f1_macro')
ax1.set_xticks(train_size)
ax1.grid()
ax2.plot(train_size, f1)
ax2.legend(['bus', 'opel', 'saab', 'van'])
ax2.title.set_text('f1_classes')
ax3.plot(train_size, accuracy)
ax3.title.set_text('accuracy')
plt.show()

In [ ]:
f1_macro, f1, accuracy = np.zeros(9), np.zeros((9, 4)), np.zeros(9)
train_size = np.zeros(9)
splits = np.linspace(0, .99, 10)
for i, split in enumerate(splits[1:]):
  # Splitting
  X_train_split, _, y_train_split, _ = train_test_split(X_train, y_train, train_size=split)
  # Best hyperparams:
  estim = RandomForestClassifier(n_estimators=32, criterion='gini')
  space = {'max_depth': range(3, 15)}
  search = GridSearchCV(estimator=estim, param_grid=space, scoring='accuracy', n_jobs=-1, refit=True)
  CV = search.fit(X_train_split, y_train_split) 
  # Fitting the model with the best hyperparams
  model = CV.best_estimator_
  model.fit(X_train_split, y_train_split)

  f1_macro[i] = f1_score(y_test, model.predict(X_test), average='macro')
  f1[i] = f1_score(y_test, model.predict(X_test), average=None)
  accuracy[i] = accuracy_score(y_test, model.predict(X_test))

  train_size[i] = split

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(28, 8)) 
ax1.plot(train_size, f1_macro)
ax1.title.set_text('f1_macro')
ax1.set_xticks(train_size)
ax1.grid()
ax2.plot(train_size, f1)
ax2.legend(['bus', 'opel', 'saab', 'van'])
ax2.title.set_text('f1_classes')
ax3.plot(train_size, accuracy)
ax3.title.set_text('accuracy')
plt.show()